In [25]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain, APIChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.schema import Document
from langchain.tools import BaseTool
from langchain.agents import ZeroShotAgent, AgentExecutor, AgentType
from functools import reduce
from langchain.chains.api import open_meteo_docs
import requests
from uuid import uuid4
import pandas as pd
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

search_endpoint = 'https://search-service01.search.windows.net'
search_key = '73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9'
private_index_name = "nois-private-v3-index"

retriever_private = SearchClient(
    endpoint=search_endpoint,
    index_name=private_index_name,
    credential=AzureKeyCredential(search_key),
    b=0.0,
    k1=0.3,
    searchMode="all"
)

In [22]:
url = "https://hrm-nois-fake.azurewebsites.net/api/User"
myobj = {
    'name': 'Bao Ho',
    'email': 'honguyenngocbao@gmail.com',
    'fullName': 'HỒ NGUYỄN NGỌC BẢO'
}

target = ''

x = requests.get(url)
print(x.json())

{'message': 'OK', 'data': [{'id': '262aa143-307a-4b9f-848d-1e6bf2e5cacb', 'name': 'Hung Bui', 'email': 'hung.bui@nois.vn', 'fullName': 'BÙI TUẤN HƯNG', 'jobTitle': 'Lập trình viên', 'mobilePhone': '0966772170', 'birthday': '1997-08-17T00:00:00', 'offDay': 0.5, 'yearOffDay': 6, 'maxDayOff': 15, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 1, 'gender': 1, 'personalEmail': 'hungbt.dev@gmail.com', 'avatar': 'https://noishrm.blob.core.windows.net/avatar/8fec9283-e82f-43ec-8f61-addb732dabcb.jpeg', 'acceptOfferDate': '2022-10-31T00:00:00', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': 'Development Unit 1', 'groupUserId': '', 'groupUserName': 'Nhân viên chính thức', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '2021-08-10T00:00:00', 'issuedBy': 'Cục trưởng CCS QLHC về TTXH', 'placeOfTemporaryResidence': '502/15/2 Thống Nhất, p.16, Gò Vấp, TP.HCM', 'placeOfResidence': '161 Đường 458, Ấp Hội Thạnh, Xã Trung A

In [3]:
# Load up your LLM
llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='test-1',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    temperature=0.5,
    max_tokens=450
)

llm2 = AzureOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='test-1',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    temperature=0,
    max_tokens=600,
    top_p=0.25,
    stop=['\n']
)

history = []
history_entry = 1

def get_history_as_txt():
    txt = ""
    hist = history[::-1]
    
    for i in hist[:3]:
        txt += f"\n<|im_start|>user\n{i['user']}\n<|im_end|>\n"
        txt += f"<|im_start|>assistant\n{i['AI']}\n<|im_end|>"
        
    return txt

def add_to_history(user_msg, ai_msg):
    history.append({'user': user_msg, 'AI': ai_msg})
    
def save_history_to_file():
    global history_entry
    file = open(f"chatbot-cases/chat-{history_entry}.txt", 'w', encoding='utf-8')
    file.write(get_history_as_txt())
    
    file.close()
    
    history_entry += 1
    
def doc_to_str(doc):
    string = ""
    for i in doc:
        string += i.page_content
        
    return string

print(llm2)
doc = open_meteo_docs.OPEN_METEO_DOCS
print(llm2.get_num_tokens(doc))

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


AzureOpenAI
Params: {'deployment_name': 'test-1', 'model_name': 'text-davinci-003', 'temperature': 0.0, 'max_tokens': 600, 'top_p': 0.25, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}, 'stop': ['\n']}
843


In [26]:
# print(open_meteo_docs.OPEN_METEO_DOCS)
hrm_doc = open("HRM_API_DOC.txt", 'r', encoding='utf-8')
api_doc = hrm_doc.read()
print(api_doc)

BASE URL: https://hrm-nois-fake.azurewebsites.net/

HRM system
API DOCUMENTATION
Endpoints:
- /api/User: does not accept any input. Use GET with this endpoint to get a list of users within the HRM system.
- /api/User/: accepts an id string as input. Use GET with thist endpoint to get a specific user using an id string. 


In [43]:
keyword_templ = """Below is a history of the conversation so far, and an input question asked by the user that needs to be answered by querying relevant company documents.
Generate a search query based on the conversation and the new question. Use Roman numerals for chapter numbers. Only include the most important queries.
Replace AND with + and OR with |. Do not answer the question.

Chat history:{context}

EXAMPLE
Input: Ai là giám đốc điều hành?
Ouput: (giám +đốc +điều +hành) | (managing +director)
Input: Ai chưa đóng tiền nước tháng 5?
Output: (tiền +nước +tháng +05) | (May +drink +fee)
Input: Was Pepsico a customer of New Ocean?
Output: Pepsico
Input: What is FASF?
Output: FASF
Input: What is the company's policy on leave?
Ouput: (ngày +nghỉ +phép) | leave
Input: Điều 7 chương 2 gồm nội dung gì?
Output: ("điều 7" + "chương II") | ("article 7" + "chapter II")

Input: {question}
Output:"""

''''''

classifier_template = """<|im_start|>system
Given a sentence, assistant will determine if the sentence belongs in 1 of 3 categories, which are:
- policy
- drink fee
- other

EXAMPLE
Input: Ai chưa đóng tiền nước tháng 5?
Output: drink fee
Input: Who has paid April 2023 Drink Fee?
Output: drink fee
Input: Quy định công ty về nơi làm việc là gì?
Output: policy
Input: What is Chapter 1, article 2 of the company policy about?
Output: policy
Input: Dịch vụ của NOIS là gì?
Output: other
Input: What is FASF?
Output: other
<|im_end|>

Input: {question}
<|im_start|>assistant
Output:"""

template = """<|im_start|>system
Assistant helps users answer their questions. Your answer must adhere to the following criteria:
- Be brief in your answers. You may use the provided sources to help answer the question. If there isn't enough information, say you don't know. If asking a clarifying question to the user would help, ask the question.
- If the user greets you, respond accordingly.
- If question is in English, answer in English. If question is in Vietnamese, answer in Vietnamese

Sources:
{summaries}
<|im_end|>

Chat history:{context}

<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

backup = '''<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Your answer must adhere to the following criteria:
- Be brief in your answers. Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
- If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
- You must answer in the question's language.

Sources:
{summaries}
<|im_end|>

Chat history:{context}

<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
'''

In [44]:
keywordChain = LLMChain(llm=llm2, prompt=PromptTemplate.from_template(keyword_templ))
qaChain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", prompt=PromptTemplate.from_template(template))
# classifierChain = LLMChain(llm=llm2, prompt=PromptTemplate.from_template(classifier_template))
# queryChain = LLMChain(llm=llm2, prompt=PromptTemplate.from_template(query_prompt_template))

In [45]:
def get_docs_using_keyword_string(keywords, ret):
    doc_lst = ret.search(search_text=keywords, top=3)
    
    doc = []
    num = 1
    for x in doc_lst:
        new_doc = Document(page_content=x['content'], metadata={'source': f"doc-{num}"})
        doc.append(new_doc)
    
        num += 1
    
    return doc

In [46]:
query = "Ánh Trần thuộc bộ phận nào?"
keyword = keywordChain({'question': query, 'context': get_history_as_txt()})['text']

print(keyword)
doc = get_docs_using_keyword_string(keyword, retriever_private)
    
result = qaChain({'input_documents': doc, 'question': query, 'context': get_history_as_txt()}, return_only_outputs=False)
print(result['output_text'])
add_to_history(query, result['output_text'])

 (Ánh +Trần) | (department +Ánh +Trần)
Ánh Trần thuộc bộ phận Data Analytics.


In [40]:
print(history)
print()
print(doc)

[{'user': 'Ánh Trần thuộc bộ phận nào?', 'AI': "I'm sorry, but I don't have enough information to answer your question. Could you please provide more context or clarify your question?"}]

[Document(page_content=' find out why Power BI is a tool that you and your business should not miss!\r\nEasy-to-remember, easy-to-use interface\r\nPower BI is designed with an easy-to-remember interface, easy to use but still effective. This is a conspicuous feature with products from Microsoft. In particular, you can customize the application’s information panel to suit your business and work data environment.\r\nPower Bi. Souce: nois.vn\r\nOptimized integration of applications to create an intuitive working environment\r\nPower BI allows users to optimally integrate diverse applications or data from many different sources, thereby providing the best support for analysis, results, and data reporting.\r\nExport reports, and analysis results quickly and safely\r\nData and reports after completing the a

In [1]:
history = []

In [2]:
string = doc_to_str(doc)

print(llm.get_num_tokens(string))
print(doc)

NameError: name 'doc_to_str' is not defined

In [ ]:
llm_water = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='test-1',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    temperature=0,
    max_tokens=300
)

query = "Tiền nước tháng 04/2023."
keyword = keywordChain({'question': query, 'context': ''})['text']
doc = get_docs_using_keyword_string(keyword, retriever_private)
print(keyword)

num = 1
for x in doc:
    x.metadata['source'] = f"doc-{num}"

    num += 1

drink = """<|im_start|>system
Given an excel spreadsheet in form of text as input, assistant calculate the number of people who has and hasn't paid their drink fee in a certain month.
The number of people who has paid their drink can be counted by counting the occurrences of the word "Done". To calculate the the number of people who hasn't paid, subtract the number of paid people from the total.
Your output will be in the following format:
- Total number of people:
- Number of people who paid:
- Number of people who hasn't paid:

Examples:
Input: \t25\tbao.nguyen@nois.vn\tNGUYỄN GIA BẢO\t17850\t17850\t\tDone\n\t26\tsinh.nguyen@nois.vn\tNGUYỄN HỮU SINH\t43800\t44000\t\tDone\n\t27\ttruong.nguyen@nois.vn\tNGUYỄN MINH TRƯỜNG\t46500\n\t28\tthien.nguyen@nois.vn\tNGUYỄN NGỌC THIỆN\t105292\t105000\t\tDone\n\t29\ttuan.nguyen@nois.vn\tNGUYỄN NGỌC TUẤN\t57950\t58000\t\tDone
Output: 
- Total number of people: 5
- Number of people who paid: 4
- Number of people who hasn't paid: 1

Input: 19\tquan.ly@nois.vn\tLÝ MINH QUÂN\t35400\t35400\t\tDone\n\t20\thuy.mach@nois.vn\tMẠCH PHƯỚC HUY\t9000\n\t21\tphong.mai@nois.vn\tMAI HOÀI PHONG\t8000\t\tChưa thu T3: 20,500\n\t22\tquan.nguyen@nois.vn\tNGUYỄN ANH QUÂN\t23950\t23950\t\tDone\n\t23\ttruyen.nguyen@nois.vn\tNGUYỄN ĐĂNG TRUYỀN\t33750\t\tChưa thu T4: 29,450\n\t24\tthin.nguyen@nois.vn\tNGUYỄN ĐÌNH THÌN\t7700
Output:
- Total number of people: 6
- Number of people who paid: 2
- Number of people who hasn't paid: 4

Input: \t34\thung.nguyen@nois.vn\tNGUYỄN VĂN HƯNG\t64400\n\t35\tan.pham@nois.vn\tPHẠM ĐẠI HOÀNG AN\t21000\n\t36\tthao.pham@nois.vn\tPHẠM HỒ NGỌC THẢO\t18750\t19000\t\tDone\n\t37\tbao.pham@nois.vn\tPHẠM HOÀNG BẢO\t47900\t48000\t\tDone\n\t38\tphuong.pham@nois.vn\tPHẠM MAI PHƯƠNG\t39500\t39500\t\tDone
Output:
- Total number of people: 5
- Number of people who paid: 3
- Number of people who hasn't paid: 2
<|im_end|>

<|im_start|>user
Input:
{summaries}
<|im_end|>
<|im_start|>assistant
Output:"""

waterChain = load_qa_with_sources_chain(llm=llm_water, chain_type="stuff", prompt=PromptTemplate.from_template(drink))
res = waterChain({'input_documents': doc, 'question': query})
res

In [34]:
print(doc)

[Document(page_content='Sheet\n\tBẢNG TỔNG HỢP TIỀN NƯỚC THÁNG 05/2023\n\tSTT\tEmail\tFullName\tMay Total\tThực thu\tNote\tTình trạng\n\t1\thung.bui@nois.vn\tBÙI TUẤN HƯNG\t68000\n\t2\thiep.dang@nois.vn\tĐẶNG DUY HIỆP\t33542\t33542\t\tDone\n\t3\ttai.dang@nois.vn\tĐẶNG HỮU TÀI\t56100\t56100\t\tDone\n\t4\tsang.dao@nois.vn\tĐÀO MINH SÁNG\t24246\t\tChưa thu T4: 5,000\n\t5\tbao.do@nois.vn\tĐỖ QUỐC BẢO\t106246\n\t6\tlong.doan@nois.vn\tĐOÀN NGỌC LONG\t31750\n\t7\tphuong.doan@nois.vn\tĐOÀN THANH PHƯỢNG\t8000\n\t8\tthong.duong@nois.vn\tDƯƠNG HOÀI THÔNG\t7000\n\t9\ttrieu.duong@nois.vn\tDƯƠNG KỲ TRIỆU\t35250\t\tChưa thu T3+4: 32,300\n\t10\tthien.duong@nois.vn\tDƯƠNG THIỆN\t8500\t8500\t\tDone\n\t11\tsang.ha@nois.vn\tHÀ CÔNG SANG\t13500\t13500\t\tDone\n\t12\thai.hoang@nois.vn\tHOÀNG THANH HẢI\t23000\t\tChưa thu T3+4: 13,400\n\t13\ttrung.hoang@nois.vn\tHOÀNG TRUNG\t38250\t38000\t\tDone\n\t14\tbao.huynh@nois.vn\tHUỲNH NGUYỄN QUỐC BẢO\t52292\n\t15\tlam.huynh@nois.vn\tHUỲNH NHỰT LÂM\t13000\t13000\t\tDo

In [36]:
query = "Số người đã đóng tiền nước tháng 5?"
keywords = keywordChain({'context': get_history_as_txt(), 'question': query})

doc = get_docs_using_keyword_string(keywords['text'], retriever_private)

num = 1
for x in doc:
    x.metadata['source'] = f"doc-{num}"

    num += 1

result = qaChain({'input_documents': doc, 'question': query, 'context': get_history_as_txt()}, return_only_outputs=False)
    
add_to_history(query, result['output_text'])

In [37]:
print(keywords['text'])

đóng tiền nước, May payment, số người, number of people


In [38]:
result['output_text']

'Tổng số người đã đóng tiền nước tháng 5 là 29 người.'

In [39]:
print(doc)

[Document(page_content='Sheet\n\tBẢNG TỔNG HỢP TIỀN NƯỚC THÁNG 05/2023\n\tSTT\tEmail\tFullName\tMay Total\tThực thu\tNote\tTình trạng\n\t1\thung.bui@nois.vn\tBÙI TUẤN HƯNG\t68000\n\t2\thiep.dang@nois.vn\tĐẶNG DUY HIỆP\t33542\t33542\t\tDone\n\t3\ttai.dang@nois.vn\tĐẶNG HỮU TÀI\t56100\t56100\t\tDone\n\t4\tsang.dao@nois.vn\tĐÀO MINH SÁNG\t24246\t\tChưa thu T4: 5,000\n\t5\tbao.do@nois.vn\tĐỖ QUỐC BẢO\t106246\n\t6\tlong.doan@nois.vn\tĐOÀN NGỌC LONG\t31750\n\t7\tphuong.doan@nois.vn\tĐOÀN THANH PHƯỢNG\t8000\n\t8\tthong.duong@nois.vn\tDƯƠNG HOÀI THÔNG\t7000\n\t9\ttrieu.duong@nois.vn\tDƯƠNG KỲ TRIỆU\t35250\t\tChưa thu T3+4: 32,300\n\t10\tthien.duong@nois.vn\tDƯƠNG THIỆN\t8500\t8500\t\tDone\n\t11\tsang.ha@nois.vn\tHÀ CÔNG SANG\t13500\t13500\t\tDone\n\t12\thai.hoang@nois.vn\tHOÀNG THANH HẢI\t23000\t\tChưa thu T3+4: 13,400\n\t13\ttrung.hoang@nois.vn\tHOÀNG TRUNG\t38250\t38000\t\tDone\n\t14\tbao.huynh@nois.vn\tHUỲNH NGUYỄN QUỐC BẢO\t52292\n\t15\tlam.huynh@nois.vn\tHUỲNH NHỰT LÂM\t13000\t13000\t\tDo

In [35]:
history = []

In [28]:
get_history_as_txt()

"\n<|im_start|>user\nWhat is New Ocean?\n<|im_end|>\n<|im_start|>assistant\nNew Ocean is a company that provides solutions for quality control in the pharmaceutical and chemical industries, as well as automation systems. They recently participated in the VIMF 2023 exhibition, with their booth located at number 76 in the Innovation area. For more information, you can contact them through their hotline at 1900 0224 or email at customercare@new-ocean.com.vn.\n<|im_end|>\n<|im_start|>user\nWhat are New Ocean's products?\n<|im_end|>\n<|im_start|>assistant\nNew Ocean provides solutions for quality control in the pharmaceutical and chemical industries, as well as automation systems. However, I do not have specific information about their products beyond this. Would you like me to look up more information?\n<|im_end|>\n<|im_start|>user\nNo, it's fine.\n<|im_end|>\n<|im_start|>assistant\nAlright, let me know if you have any other questions related to New Ocean or their products.\n<|im_end|>\n<|

In [29]:
save_history_to_file()

In [33]:
history = []

In [31]:
doc_txt = ""
for x in doc:
    doc_txt += x.page_content
    
context = get_history_as_txt()
    
msg = f"""<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
Sources:
{doc_txt}
<|im_end|>
{context}
"""

print(msg)
print(llm.get_num_tokens(msg))
print(f"Length of chat history: {llm.get_num_tokens(get_history_as_txt())}")

<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
Sources:
n viên giàu kinh nghiệm là đơn vị luôn nắm bắt nhanh chóng các xu hướng công nghệ mới. NOIS đưa sự phát triển của công nghệ và chuyển đổi số vào các lĩnh vực sản xuất một cách thông minh, đúng nơi, đúng việc. Nổi tiếng với hơn 70+ đối tác là các thương hiệu nổi tiếng như Vinamilk, Dutch LaDy, Amway, Nestle, P&G, HTL… NOIS has affirmed its position in providing solution packages to support optimal business managem

In [7]:
class PrivateDataTool(BaseTool):
    name = "Excel sheet getter tool"
    description = """use this tool when you need to get an information sheet from an excel file exclusive only to New Ocean and NOIS's employees.
    Query by inputting the sheet name. The excel file contains 6 sheets (case-sensitive) which are:
    - Training Plan: contains past and upcoming company training plans.
    - Company Trip_Team Building 2023: contains upcoming company trips.
    - Organization Chart: contains details regarding development teams, members, their positions and their managers.
    - May Drink Fee 2023: contains info on people who have or haven't paid drink fee in May 2023.
    - April Drink Fee 2023: contains info on people who have or haven't paid drink fee in April 2023.
    - March Drink Fee 2023: contains info on people who have or haven't paid drink fee in March 2023."""

    def _run(self, query: str):
        try:
            df = pd.read_excel("Data Private Chatbot.xlsx", sheet_name=query, skiprows=1)
            return df
        
        except Exception:
            return f"\nSheet {query} not found, the file contains only 4 sheets which are Training Plan, Organization Chart, Company Trip_Team Building 2023 and May Drink Fee. Try inputting the correct sheet name."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")
        
class CountTool(BaseTool):
    name = "Excel sheet item counter tool"
    description = """DO NOT use this tool if the user asks you to return names or a list. Use this tool for counting the number of times an item appears within a column in an excel sheet.
    Input is a string containing 3 parameters for sheet name, column name and query value (if the query value is empty or nan/NaN, use <empty>), respectively and separated by a comma and a space. Example:
    - May Drink Fee 2023, Tình trạng, Done
    - April Drink Fee 2023, Tình trạng, <empty>"""
    
    def _run(self, query: str):
        lst = query.split(', ')
        df = pd.read_excel("Data Private Chatbot.xlsx", sheet_name=lst[0], skiprows=1)
        
        try:
            unique_val = df[lst[1]].unique().tolist()
        except Exception as e:
            cols = df.columns.values
            string = ""

            for x in cols:
                string += x
                if x != cols[-1]:
                    string += ', '
            
            return f"Column {e} does not exist, valid columns are: {string}"
        
        get_empty = True if lst[2] == "<empty>" else False
                
        if lst[2] not in unique_val and not get_empty:
            return f"Incorrect filter value '{lst[2]}', column only contains these filter values: {unique_val}\nTo filter nan values, use <empty>."
        
        if not get_empty:
            return f"{len(df[df[lst[1]] == lst[2]])}. Use this result over any of your previous numeric result(s)."
        
        return f"{len(df[df[lst[1]].isnull() == True])}. Use this result over any of your previous numeric result(s)."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")
        
class FilterTool(BaseTool):
    name = "Excel sheet filter tool"
    description = """use this tool if you want to filter a sheet when you know the column name and the filter value. Sheets in this file do not contain any duplicates.
    Input is a string containing 3 parameters for sheet name, column name and query value (if the query value is empty or nan/NaN, use <empty>), respectively and separated by a comma and a space. Example:
    - May Drink Fee 2023, Tình trạng, Done
    - April Drink Fee 2023, Tình trạng, <empty>"""
    
    def _run(self, query: str):
        lst = query.split(', ')
        df = pd.read_excel("Data Private Chatbot.xlsx", sheet_name=lst[0], skiprows=1)
        
        try:
            unique_val = df[lst[1]].unique().tolist()
        except Exception as e:
            cols = df.columns.values
            string = ""

            for x in cols:
                string += x
                if x != cols[-1]:
                    string += ', '
            
            return f"Column {e} does not exist, valid columns are: {string}"
        
        get_empty = True if lst[2] == "<empty>" else False
                
        if lst[2] not in unique_val and not get_empty:
            return f"Incorrect filter value '{lst[2]}', column only contains these filter values: {unique_val}\nTo filter nan values, use <empty>."
        
        string = ""
        res = df[df[lst[1]].isnull() == True] if get_empty else df[df[lst[1]] == lst[2]]
        
#         num = 0
#         for x in res.values:
#             string += f"- {x[2]}\n"
#             num += 1
        
#         return string + f"Total: {num}"
        return res
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

prefix = """You are an intelligent assistant helping New Ocean and NOIS employees with company related questions. You will be provided with keywords that you can use.
If you cannot answer by using only 1 tool, try using other provided tools. If you cannot get any results from tools, say you don't know. Do not use any other sources other than the ones you obtain from tools.
Make sure to print out the result and answer in the user's language.
You have access to the following tools:"""

suffix = """Begin!

Keywords: {keywords}
Answer in the user's language. The user cannot see your observation. If the user asks for a list or names and you return a number result, you did something wrong.
Question: {input}
{agent_scratchpad}"""

tools = [PrivateDataTool(), FilterTool(), CountTool()]

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["keywords", "input", "agent_scratchpad"],
)

llm_chain = LLMChain(llm=llm2, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True, stop=["\nObservation:", "<|im_end|>"])
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors="Check your output and make sure it conforms! Start your answer with Final Answer:"
)

query = "Tên những người đã trả tiền nước tháng 3?"
keywords = keywordChain({'question': query, 'context': ''})['text']
print(keywords)

res = agent_chain.run(input=query, keywords=keywords)
res

trả tiền nước tháng 03, March drink fee, những người, people


> Entering new  chain...
Thought: I need to use the Excel sheet filter tool to filter the "March Drink Fee 2023" sheet by the "Tình trạng" column with a query value of "Done".
Action: Excel sheet filter tool
Action Input: March Drink Fee 2023, Tình trạng, Done
Observation:     STT                       Email               FullName  Macrh Total  \
0     1            hung.bui@nois.vn          BÙI TUẤN HƯNG        76300   
1     2           hiep.dang@nois.vn          ĐẶNG DUY HIỆP        28750   
2     3            tai.dang@nois.vn           ĐẶNG HỮU TÀI        17200   
3     4            sang.dao@nois.vn          ĐÀO MINH SÁNG        35700   
4     5           minh.dinh@nois.vn        ĐINH QUANG MINH         4500   
5     6              nam.do@nois.vn            ĐỖ NGỌC NAM        21500   
6     7              bao.do@nois.vn            ĐỖ QUỐC BẢO        28150   
7     8           long.doan@nois.vn         ĐOÀN NGỌC LONG     

'Những người đã trả tiền nước tháng 3 là các nhân viên trong danh sách trên.'

In [8]:
# # Define query tool for agent
# class DocumentQueryTool(BaseTool):
#     name = "Document query tool"
#     description = "use this tool when you need to get documents for questions related to New Ocean and NOIS."
    
#     def _run(self, query: str):
#         doc = retriever_public.get_relevant_documents(query)[:4]
#         try:
#             content = reduce(lambda x, y: x + y.page_content, doc, doc[0].page_content)
#             return content
#         except Exception:
#             return "No documents found."
    
#     def _arun(self, query: str):
#         raise NotImplementedError("This tool does not support async")